In [3]:
import os
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()

In [4]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data/lyrics')

In [5]:
text_all = ''
for song in os.listdir(data_dir):
    if song.endswith('.txt'):
        text = open(os.path.join(data_dir, song), mode = 'rb').read().decode(encoding = "utf-8")
        text_all += text

In [6]:
vocab = sorted(set(text_all))
print ('{} unique characters'.format(len(vocab)))

2007 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text_all])

In [8]:
seq_length = 100
examples_per_epoch = len(text_all) // seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

sequences = char_dataset.batch(seq_length + 1, drop_remainder = True)
dataset = sequences.map(split_input_target)

In [10]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch // BATCH_SIZE
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [11]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024 

In [12]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
    rnn2 = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation = 'sigmoid')

In [13]:
def build_model(vocab_size, embedding_dim, lstm_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
        batch_input_shape = [batch_size, None]),
    rnn(rnn_units,
        return_sequences = True, 
        recurrent_initializer = 'glorot_uniform',
        stateful = True),
#     rnn(rnn_units,
#         return_sequences = True, 
#         recurrent_initializer = 'glorot_uniform',
#         stateful = True),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [15]:
model = build_model(
    vocab_size = len(vocab), 
    embedding_dim = embedding_dim, 
    lstm_units = rnn_units, 
    batch_size = BATCH_SIZE)

In [16]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

In [17]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [18]:
checkpoint_dir = './Lyrics_training'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

In [19]:
EPOCHS = 50

In [20]:
history = model.fit(dataset.repeat(), epochs = EPOCHS, steps_per_epoch = steps_per_epoch, callbacks = [checkpoint_callback])

Epoch 1/50
4/4 [==============================] - 18s 5s/step - loss: 7.5677
Epoch 2/50
4/4 [==============================] - 19s 5s/step - loss: 6.1845
Epoch 3/50
4/4 [==============================] - 19s 5s/step - loss: 5.8588
Epoch 4/50
4/4 [==============================] - 19s 5s/step - loss: 5.7655
Epoch 5/50
4/4 [==============================] - 21s 5s/step - loss: 5.6776
Epoch 6/50
4/4 [==============================] - 20s 5s/step - loss: 5.5701
Epoch 7/50
4/4 [==============================] - 21s 5s/step - loss: 5.5009
Epoch 8/50
4/4 [==============================] - 20s 5s/step - loss: 5.4603
Epoch 9/50
4/4 [==============================] - 19s 5s/step - loss: 5.4131
Epoch 10/50
4/4 [==============================] - 19s 5s/step - loss: 5.3759
Epoch 11/50
4/4 [==============================] - 19s 5s/step - loss: 5.3108
Epoch 12/50
4/4 [==============================] - 19s 5s/step - loss: 5.2346
Epoch 13/50
4/4 [==============================] - 19s 5s/step - loss: 5.

In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training\\ckpt_50'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            513792    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 2007)           2057175   
Total params: 6,506,199
Trainable params: 6,506,199
Non-trainable params: 0
_________________________________________________________________


In [33]:
def generate_text(model, start_string, temperature = 1, num_generate = 100):
    
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))